In [ ]:
#!/usr/bin/env python3
import os
import glob
import re
import pandas as pd

# ─── CONFIG ───────────────────────────────────────────────────────────────────

# Folder where your individual inference TSVs live:
PRED_DIR = "../datasets"
# Glob pattern to match all your teste100_*_predictions.tsv files:
PATTERN  = os.path.join(PRED_DIR, "teste100_*_predictions.tsv")

# ─── MERGE ────────────────────────────────────────────────────────────────────

files = sorted(glob.glob(PATTERN))
if not files:
    print("❌ No prediction files found:", PATTERN)
    exit(1)

merged_df = None

for path in files:
    fname = os.path.basename(path)
    m = re.match(r"teste100_(.*)_predictions\.tsv$", fname)
    if not m:
        print(f"⚠️  Skipping unexpected file name: {fname}")
        continue
    model_name = m.group(1)

    # load
    df = pd.read_csv(path, sep="\t", dtype=str)
    # initialize on first file
    if merged_df is None:
        merged_df = df[["text", "label(1=real,0=fake)"]].copy()

    # add columns
    merged_df[f"{model_name}_predicted"]      = df["predicted"]
    # some older scripts may have None or missing prob column
    if "predicted_prob" in df.columns:
        merged_df[f"{model_name}_predicted_prob"] = df["predicted_prob"]
    else:
        merged_df[f"{model_name}_predicted_prob"] = ""

# ─── WRITE OUT ────────────────────────────────────────────────────────────────

out_path = "../datasets/merged_teste100_predictions.tsv"
merged_df.to_csv(out_path, sep="\t", index=False)
print(f"✅ Merged {len(files)} files into {out_path}")


✅ Merged 24 files into merged_teste100_predictions.tsv


In [4]:
#!/usr/bin/env python3
import pandas as pd

# ─── CONFIG ───────────────────────────────────────────────────────────────────

MERGED_TSV = "../datasets/merged_teste100_predictions.tsv"

# ─── LOAD & SETUP ─────────────────────────────────────────────────────────────

df = pd.read_csv(MERGED_TSV, sep="\t", dtype=str)

# Identify the prediction columns (everything ending with "_predicted")
pred_cols = [c for c in df.columns if c.endswith("_predicted")]

# Normalize the true label to "REAL"/"FAKE"
df["true_label"] = (
    df["label(1=real,0=fake)"]
      .str.strip()
      .map({"1":"REAL","__LABEL__1":"REAL","REAL":"REAL",
            "0":"FAKE","__LABEL__0":"FAKE","FAKE":"FAKE"})
)

# Compute a “correct” boolean for each model
for col in pred_cols:
    df[col + "_correct"] = (df[col] == df["true_label"])

# Sum how many models got each row right
correct_cols = [c for c in df.columns if c.endswith("_correct")]
df["correct_count"] = df[correct_cols].sum(axis=1).astype(int)

# Total number of models
n_models = len(pred_cols)

# ─── FIND THE WORST ARTICLE PER CLASS ─────────────────────────────────────────

def report_worst(label):
    sub = df[df["true_label"] == label]
    # find the minimum correct_count
    min_corr = sub["correct_count"].min()
    worst = sub[sub["correct_count"] == min_corr].iloc[0]
    models_correct = [col.replace("_predicted_correct","") 
                      for col in correct_cols 
                      if worst[col]]
    print(f"\n--- Worst {label} article (only {min_corr}/{n_models} models got it right) ---")
    print(f"Text: {worst['text']}\n")
    print("Models that guessed it correctly:")
    print("\n".join(f"  • {m}" for m in models_correct))

report_worst("FAKE")
report_worst("REAL")



--- Worst FAKE article (only 8/24 models got it right) ---
Text: The @DOGE team just discovered that FEMA sent $59M LAST WEEK to luxury hotels in New York City to house illegal migrants. Sending this money violated the law and is in gross insubordination to the President’s executive order. That money is meant for American disaster relief and instead is being spent on high end hotels for illegals! A clawback demand will be made today to recoup those funds.

Models that guessed it correctly:
  • KNN_GloVe_300d
  • LSTM_GloVe
  • LSTM
  • RandomForest_GloVe_300d
  • SVM_Bag_of_Words_(1-3gram)
  • bert_v2
  • roberta_adversarial
  • roberta_v3

--- Worst REAL article (only 4/24 models got it right) ---
Text: Democrats mistakenly tweet 2014 pictures from Obama’s term showing children from the Border in steel cages. They thought it was recent pictures in order to make us look bad, but backfires. Dems must agree to Wall and new Border Protection for good of country...Bipartisan Bill!

Models

In [5]:
#!/usr/bin/env python3
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# ─── CONFIG ───────────────────────────────────────────────────────────────────


# ─── LOAD ─────────────────────────────────────────────────────────────────────

df = pd.read_csv(MERGED_TSV, sep="\t", dtype=str)

# identify all the model‐prediction columns
models = [c[:-10]  # strip "_predicted"
          for c in df.columns 
          if c.endswith("_predicted")]

y_true = df["label(1=real,0=fake)"].str.strip().map({
    "1":"REAL","__LABEL__1":"REAL","REAL":"REAL",
    "0":"FAKE","__LABEL__0":"FAKE","FAKE":"FAKE"
}).tolist()

# ─── COMPUTE & DISPLAY ───────────────────────────────────────────────────────

print(f"{'Model':25} | {'Acc':>6} | {'Prec_G':>7} | {'Rec_G':>7} | {'F1_G':>7} |"
      f" {'Prec_FAKE':>9} | {'Rec_FAKE':>9} | {'F1_FAKE':>7} |"
      f" {'Prec_REAL':>9} | {'Rec_REAL':>9} | {'F1_REAL':>7}")
print("-"*110)

for m in models:
    y_pred = df[f"{m}_predicted"].tolist()
    # general (macro)
    acc_g   = accuracy_score(y_true, y_pred)
    prec_g  = precision_score(y_true, y_pred, average="macro", zero_division=0)
    rec_g   = recall_score   (y_true, y_pred, average="macro", zero_division=0)
    f1_g    = f1_score       (y_true, y_pred, average="macro", zero_division=0)
    # fake as positive
    prec_f  = precision_score(y_true, y_pred, pos_label="FAKE", zero_division=0)
    rec_f   = recall_score   (y_true, y_pred, pos_label="FAKE", zero_division=0)
    f1_f    = f1_score       (y_true, y_pred, pos_label="FAKE", zero_division=0)
    # real as positive
    prec_r  = precision_score(y_true, y_pred, pos_label="REAL", zero_division=0)
    rec_r   = recall_score   (y_true, y_pred, pos_label="REAL", zero_division=0)
    f1_r    = f1_score       (y_true, y_pred, pos_label="REAL", zero_division=0)

    print(f"{m:25} |"
          f" {acc_g*100:5.2f}% |"
          f" {prec_g*100:6.2f}% |"
          f" {rec_g*100:6.2f}% |"
          f" {f1_g*100:6.2f}% |"
          f" {prec_f*100:8.2f}% |"
          f" {rec_f*100:8.2f}% |"
          f" {f1_f*100:6.2f}% |"
          f" {prec_r*100:8.2f}% |"
          f" {rec_r*100:8.2f}% |"
          f" {f1_r*100:6.2f}%")


Model                     |    Acc |  Prec_G |   Rec_G |    F1_G | Prec_FAKE |  Rec_FAKE | F1_FAKE | Prec_REAL |  Rec_REAL | F1_REAL
--------------------------------------------------------------------------------------------------------------
CNN_GloVe                 | 84.00% |  85.42% |  84.00% |  83.84% |    78.33% |    94.00% |  85.45% |    92.50% |    74.00% |  82.22%
CNN_LSTM_GloVe            | 82.00% |  84.72% |  82.00% |  81.64% |    75.00% |    96.00% |  84.21% |    94.44% |    68.00% |  79.07%
CNN_LSTM                  | 82.00% |  84.72% |  82.00% |  81.64% |    75.00% |    96.00% |  84.21% |    94.44% |    68.00% |  79.07%
CNN                       | 84.00% |  85.42% |  84.00% |  83.84% |    78.33% |    94.00% |  85.45% |    92.50% |    74.00% |  82.22%
KNN_Bag_of_Words_(1-3gram) | 60.00% |  60.15% |  60.00% |  59.86% |    61.36% |    54.00% |  57.45% |    58.93% |    66.00% |  62.26%
KNN_GloVe_300d            | 56.00% |  56.37% |  56.00% |  55.36% |    57.89% |    44.00% |

In [7]:
#!/usr/bin/env python3
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# ─── CONFIG ───────────────────────────────────────────────────────────────────


# ─── 1) Load merged predictions ────────────────────────────────────────────────

df = pd.read_csv(MERGED_TSV, sep="\t", dtype=str)

# identify model names from the "_predicted" columns
model_names = [c[:-10] for c in df.columns if c.endswith("_predicted")]

# normalize true labels
y_true = df["label(1=real,0=fake)"].str.strip().map({
    "1":"REAL","__LABEL__1":"REAL","REAL":"REAL",
    "0":"FAKE","__LABEL__0":"FAKE","FAKE":"FAKE"
}).tolist()

# ─── 2) Compute all metrics into a DataFrame ──────────────────────────────────

records = []
for m in model_names:
    y_pred = df[f"{m}_predicted"].tolist()
    # general (macro-average)
    acc_g   = accuracy_score(y_true, y_pred)
    prec_g  = precision_score(y_true, y_pred, average="macro", zero_division=0)
    rec_g   = recall_score   (y_true, y_pred, average="macro", zero_division=0)
    f1_g    = f1_score       (y_true, y_pred, average="macro", zero_division=0)
    # FAKE as positive
    prec_f  = precision_score(y_true, y_pred, pos_label="FAKE", zero_division=0)
    rec_f   = recall_score   (y_true, y_pred, pos_label="FAKE", zero_division=0)
    f1_f    = f1_score       (y_true, y_pred, pos_label="FAKE", zero_division=0)
    # REAL as positive
    prec_r  = precision_score(y_true, y_pred, pos_label="REAL", zero_division=0)
    rec_r   = recall_score   (y_true, y_pred, pos_label="REAL", zero_division=0)
    f1_r    = f1_score       (y_true, y_pred, pos_label="REAL", zero_division=0)

    records.append({
        "model":     m,
        "acc_gen":   acc_g,
        "prec_gen":  prec_g,
        "rec_gen":   rec_g,
        "f1_gen":    f1_g,
        "prec_fake": prec_f,
        "rec_fake":  rec_f,
        "f1_fake":   f1_f,
        "prec_real": prec_r,
        "rec_real":  rec_r,
        "f1_real":   f1_r,
    })

dfm = pd.DataFrame(records).set_index("model")

# ─── 3) Define human-friendly titles for each metric ──────────────────────────

metrics = {
    "Accuracy (general)":     "acc_gen",
    "Precision (macro-avg)":  "prec_gen",
    "Recall (macro-avg)":     "rec_gen",
    "F1 (macro-avg)":         "f1_gen",
    "Precision (FAKE)":       "prec_fake",
    "Recall (FAKE)":          "rec_fake",
    "F1 (FAKE)":              "f1_fake",
    "Precision (REAL)":       "prec_real",
    "Recall (REAL)":          "rec_real",
    "F1 (REAL)":              "f1_real",
}

# ─── 4) For each metric, print Top 5 and Bottom 5 ──────────────────────────────

for title, col in metrics.items():
    s = dfm[col].sort_values(ascending=False)
    print(f"\n🏆 Top 5 by {title}:")
    for model, val in s.head(5).items():
        print(f"  • {model:30} {val*100:6.2f}%")
    print(f"\n🚩 Bottom 5 by {title}:")
    for model, val in s.tail(5).items():
        print(f"  • {model:30} {val*100:6.2f}%")



🏆 Top 5 by Accuracy (general):
  • SVM_GloVe_300d                  95.00%
  • roberta_v3                      92.00%
  • roberta_adversarial             91.00%
  • LSTM_GloVe                      90.00%
  • bert_v2                         90.00%

🚩 Bottom 5 by Accuracy (general):
  • SVM_TFIDF_(1-3gram)             56.00%
  • KNN_TFIDF_(1-3gram)             55.00%
  • LogisticRegression_Bag_of_Words_(1-3gram)  55.00%
  • NaiveBayes_TFIDF_(1-3gram)      50.00%
  • NaiveBayes_Bag_of_Words_(1-3gram)  50.00%

🏆 Top 5 by Precision (macro-avg):
  • SVM_GloVe_300d                  95.02%
  • roberta_v3                      92.07%
  • roberta_adversarial             91.41%
  • LSTM_GloVe                      91.05%
  • LSTM                            91.05%

🚩 Bottom 5 by Precision (macro-avg):
  • KNN_Bag_of_Words_(1-3gram)      60.15%
  • KNN_TFIDF_(1-3gram)             57.06%
  • KNN_GloVe_300d                  56.37%
  • NaiveBayes_TFIDF_(1-3gram)      25.00%
  • NaiveBayes_Bag_of_Words_(

In [1]:
import os
import glob
import pandas as pd

In [2]:
PRED_DIR = "../datasets"

# Pattern to match
PATTERN = os.path.join(PRED_DIR, "*_predictions.tsv")

# How many top errors to display
TOP_K = 5

In [13]:
files = sorted(glob.glob(PATTERN))
print("Found prediction files:", files)

dfs = []
model_names = []
for path in files:
    df = pd.read_csv(path, sep="\t", dtype={"label(1=real,0=fake)": str})
    print(f"  → {os.path.basename(path)}: {len(df)} rows before cleaning")

    # count any missing text
    n_missing = df["text"].isna().sum()
    if n_missing:
        print(f"    ⚠️  {n_missing} rows with missing text")

    basename = os.path.basename(path)
    model = basename.replace("_predictions.tsv","")
    model_names.append(model)
    df = df.rename(columns={
        "predicted":      f"pred_{model}",
        "predicted_prob": f"prob_{model}",
        "label(1=real,0=fake)": "true_raw"
    })
    dfs.append(df[["text", "true_raw", f"pred_{model}"]])

# merge
merged = dfs[0][["text", "true_raw"]].copy()
for df in dfs:
    merged = pd.concat([merged, df.drop(columns=["text","true_raw"])], axis=1)

print(f"\nAfter concatenation: {len(merged)} rows total")

# normalize and drop
merged["true_label"] = merged["true_raw"].str.strip().map({
    "1":"REAL","__LABEL__1":"REAL","REAL":"REAL",
    "0":"FAKE","__LABEL__0":"FAKE","FAKE":"FAKE"
})
before_drop = len(merged)
merged = merged.dropna(subset=["true_label"])
after_drop = len(merged)
print(f"Dropped {before_drop-after_drop} rows because true_label was NaN (now {after_drop} rows)\n")

# sanity check
print("Unique true_label values:", merged["true_label"].value_counts().to_dict())


Found prediction files: ['../datasets/teste100_CNN_LSTM_predictions.tsv', '../datasets/teste100_CNN_predictions.tsv', '../datasets/teste100_KNN_Bag_of_Words_(1-3gram)_predictions.tsv', '../datasets/teste100_KNN_GloVe_300d_predictions.tsv', '../datasets/teste100_KNN_TFIDF_(1-3gram)_predictions.tsv', '../datasets/teste100_LSTM_predictions.tsv', '../datasets/teste100_LogisticRegression_Bag_of_Words_(1-3gram)_predictions.tsv', '../datasets/teste100_LogisticRegression_GloVe_300d_predictions.tsv', '../datasets/teste100_LogisticRegression_TFIDF_(1-3gram)_predictions.tsv', '../datasets/teste100_NaiveBayes_Bag_of_Words_(1-3gram)_predictions.tsv', '../datasets/teste100_NaiveBayes_GloVe_300d_predictions.tsv', '../datasets/teste100_NaiveBayes_TFIDF_(1-3gram)_predictions.tsv', '../datasets/teste100_RandomForest_Bag_of_Words_(1-3gram)_predictions.tsv', '../datasets/teste100_RandomForest_GloVe_300d_predictions.tsv', '../datasets/teste100_RandomForest_TFIDF_(1-3gram)_predictions.tsv', '../datasets/tes

In [14]:
#  ———  rank by fake‐news accuracy  ———

# mask for all the true‐fake examples
fake_mask = merged["true_label"] == "FAKE"
total_fakes = fake_mask.sum()

fake_accs = {}
for m in model_names:
    # count how many of the FAKEs each model got right
    correct = (merged.loc[fake_mask, f"pred_{m}"] == "FAKE").sum()
    fake_accs[m] = correct / total_fakes

# sort descending
ranked = sorted(fake_accs.items(), key=lambda x: x[1], reverse=True)

print("\n📊 Fake‐news detection accuracy (best → worst):")
for name, acc in ranked:
    print(f"  {name:<40} {acc*100:5.2f}%")



📊 Fake‐news detection accuracy (best → worst):
  teste100_LSTM                            98.00%
  teste100_CNN_LSTM                        96.00%
  teste100_SVM_GloVe_300d                  96.00%
  teste100_CNN                             94.00%
  teste100_bert_v2                         94.00%
  teste100_roberta_v3                      90.00%
  teste100_with                            86.00%
  teste100_fasttext_cnn_lstm               82.00%
  teste100_LogisticRegression_GloVe_300d   76.00%
  teste100_RandomForest_GloVe_300d         76.00%
  teste100_RandomForest_TFIDF_(1-3gram)    76.00%
  teste100_RandomForest_Bag_of_Words_(1-3gram) 70.00%
  teste100_NaiveBayes_GloVe_300d           58.00%
  teste100_KNN_Bag_of_Words_(1-3gram)      54.00%
  teste100_SVM_Bag_of_Words_(1-3gram)      50.00%
  teste100_KNN_GloVe_300d                  44.00%
  teste100_KNN_TFIDF_(1-3gram)             28.00%
  teste100_LogisticRegression_TFIDF_(1-3gram) 24.00%
  teste100_SVM_TFIDF_(1-3gram)             14

In [15]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# -----------------------------------------------------------------------------
# 1) Build a small metrics table: rows=models, cols=metrics
# -----------------------------------------------------------------------------

metrics = []
y_true = merged["true_label"].tolist()

for m in model_names:
    y_pred = merged[f"pred_{m}"].tolist()
    # general
    acc_gen = accuracy_score(y_true, y_pred)
    prec_gen = precision_score(y_true, y_pred, average="macro", zero_division=0)
    rec_gen  = recall_score   (y_true, y_pred, average="macro", zero_division=0)
    f1_gen   = f1_score       (y_true, y_pred, average="macro", zero_division=0)
    # fake as positive
    prec_fake = precision_score(y_true, y_pred, pos_label="FAKE", zero_division=0)
    rec_fake  = recall_score   (y_true, y_pred, pos_label="FAKE", zero_division=0)
    f1_fake   = f1_score       (y_true, y_pred, pos_label="FAKE", zero_division=0)
    # real as positive
    prec_real = precision_score(y_true, y_pred, pos_label="REAL", zero_division=0)
    rec_real  = recall_score   (y_true, y_pred, pos_label="REAL", zero_division=0)
    f1_real   = f1_score       (y_true, y_pred, pos_label="REAL", zero_division=0)

    metrics.append({
        "model":      m,
        # general
        "acc_gen":    acc_gen,
        "prec_gen":   prec_gen,
        "rec_gen":    rec_gen,
        "f1_gen":     f1_gen,
        # fake
        "prec_fake":  prec_fake,
        "rec_fake":   rec_fake,
        "f1_fake":    f1_fake,
        # real
        "prec_real":  prec_real,
        "rec_real":   rec_real,
        "f1_real":    f1_real,
    })

import pandas as pd
dfm = pd.DataFrame(metrics).set_index("model")

# -----------------------------------------------------------------------------
# 2) A helper to print Top-10 for any column
# -----------------------------------------------------------------------------
def print_top(col, title):
    print(f"\n🏆 Top-10 by {title}")
    top = dfm[col].sort_values(ascending=False).head(10)
    # pretty-print
    print(top.to_frame(name=title).to_markdown())

# -----------------------------------------------------------------------------
# 3) Print all Twelve lists
# -----------------------------------------------------------------------------

# General
print_top("acc_gen",   "Accuracy (general)")
print_top("prec_gen",  "Precision (macro)")
print_top("rec_gen",   "Recall (macro)")
print_top("f1_gen",    "F1 (macro)")

# Fake class
print_top("prec_fake", "Precision (FAKE)")
print_top("rec_fake",  "Recall (FAKE)")
print_top("f1_fake",   "F1 (FAKE)")

# Real class
print_top("prec_real", "Precision (REAL)")
print_top("rec_real",  "Recall (REAL)")
print_top("f1_real",   "F1 (REAL)")



🏆 Top-10 by Accuracy (general)
| model                                  |   Accuracy (general) |
|:---------------------------------------|---------------------:|
| teste100_SVM_GloVe_300d                |                 0.95 |
| teste100_roberta_v3                    |                 0.92 |
| teste100_with                          |                 0.91 |
| teste100_bert_v2                       |                 0.9  |
| teste100_LSTM                          |                 0.9  |
| teste100_LogisticRegression_GloVe_300d |                 0.88 |
| teste100_CNN                           |                 0.84 |
| teste100_RandomForest_GloVe_300d       |                 0.83 |
| teste100_CNN_LSTM                      |                 0.82 |
| teste100_RandomForest_TFIDF_(1-3gram)  |                 0.81 |

🏆 Top-10 by Precision (macro)
| model                                  |   Precision (macro) |
|:---------------------------------------|--------------------:|
| teste100_SVM_

In [11]:
y_true

['FAKE',
 'REAL',
 'FAKE',
 'FAKE',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'FAKE',
 'REAL',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE',
 'FAKE']